In [ ]:
# baseline model


# 4 ways to improve the model

# 1. Increase training data size

# 2. Add more features to the training data

# 3. Try different algorithms

# 4. Tune the hyperparameters of the model



In [1]:
import pandas as pd

# Load the data
data = pd.read_parquet('../data/transformed/yellow_tripdata_features_target.parquet')

In [2]:
data.head()

data[data['PULocationID'] == 43].head()

,rides_previous_672,rides_previous_671,rides_previous_670,rides_previous_669,rides_previous_668,rides_previous_667,rides_previous_666,rides_previous_665,rides_previous_664,rides_previous_663,...,rides_previous_7,rides_previous_6,rides_previous_5,rides_previous_4,rides_previous_3,rides_previous_2,rides_previous_1,pickup_hour,PULocationID,target_rides_next_hour
24564,273.0,262.0,79.0,22.0,11.0,11.0,11.0,32.0,28.0,67.0,...,251.0,250.0,126.0,160.0,99.0,91.0,68.0,2019-01-29,43,NaN
24565,17.0,5.0,2.0,3.0,8.0,14.0,44.0,91.0,109.0,159.0,...,258.0,260.0,168.0,143.0,86.0,83.0,55.0,2019-01-30,43,NaN
24566,19.0,12.0,6.0,2.0,5.0,26.0,46.0,96.0,102.0,174.0,...,259.0,263.0,129.0,124.0,103.0,83.0,58.0,2019-01-31,43,NaN
24567,20.0,14.0,6.0,4.0,2.0,31.0,26.0,109.0,131.0,181.0,...,287.0,245.0,140.0,140.0,126.0,106.0,72.0,2019-02-01,43,NaN
24568,37.0,17.0,7.0,4.0,7.0,14.0,20.0,61.0,78.0,170.0,...,315.0,271.0,177.0,202.0,132.0,114.0,116.0,2019-02-02,43,NaN


In [23]:
from datetime import datetime
from typing import Tuple
import pandas as pd

def create_training_sets(PATH, cutoff_date, target_column_name): 
    """
    Generate separate training and testing sets for each PULocationID.
    
    :param PATH: Path to the data file.
    :param cutoff_date: The cutoff date for splitting into train/test sets.
    :param target_column_name: The name of the target column.
    :return: A dictionary where each PULocationID has its (X_train, X_test, y_train, y_test).
    """
    data = pd.read_parquet(PATH)
    
    # Dictionary to store training/testing sets for each PULocationID
    training_sets = {}

    for pulocation_id in data['PULocationID'].unique():
        print(f"Processing PULocationID: {pulocation_id}")
        
        # Filter data specific to each PULocationID
        location_data = data[data['PULocationID'] == pulocation_id].copy()
        location_data.drop(columns=['PULocationID'], inplace=True)
        # Perform train/test split for this location
        X_train, X_test, y_train, y_test = train_test_split(
            location_data,
            cutoff_date,
            target_column_name
        )

        # Store the result in the dictionary
        training_sets[pulocation_id] = (X_train, X_test, y_train, y_test)

    return training_sets


In [24]:
# test the function
training_sets = create_training_sets(
    '../data/transformed/yellow_tripdata_features_target.parquet',
    datetime(2019, 6, 1),
    'target'
)

Processing PULocationID: 4
Processing PULocationID: 7
Processing PULocationID: 10
Processing PULocationID: 11
Processing PULocationID: 12
Processing PULocationID: 13
Processing PULocationID: 14
Processing PULocationID: 17
Processing PULocationID: 18
Processing PULocationID: 19
Processing PULocationID: 20
Processing PULocationID: 21
Processing PULocationID: 24
Processing PULocationID: 25
Processing PULocationID: 33
Processing PULocationID: 34
Processing PULocationID: 35
Processing PULocationID: 36
Processing PULocationID: 37
Processing PULocationID: 39
Processing PULocationID: 40
Processing PULocationID: 41
Processing PULocationID: 42
Processing PULocationID: 43
Processing PULocationID: 45
Processing PULocationID: 48
Processing PULocationID: 49
Processing PULocationID: 50
Processing PULocationID: 51
Processing PULocationID: 52
Processing PULocationID: 54
Processing PULocationID: 56
Processing PULocationID: 60
Processing PULocationID: 61
Processing PULocationID: 63
Processing PULocationI

In [ ]:
# for each pulocationid, we have a tuple of X_train, X_test, y_train, y_test
# now, we can train a model for each pulocationid
# we should create a function to train a model for each pulocationid

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split

def train_model(model, X_train, y_train):
    """
    Train a machine learning model.
    
    :param X_train: The training features.
    :param y_train: The training target.
    :return: The trained model.
    """
    model.fit(X_train, y_train)
    return model

def evaluate_model(model, X_test, y_test):
    """
    Evaluate a machine learning model.
    
    :param model: The trained model.
    :param X_test: The testing features.
    :param y_test: The testing target.
    :return: The evaluation metric.
    """
    y_pred = model.predict(X_test)
    return root_mean_squared_error(y_test, y_pred)
